In [1]:

import numpy as np
import zipfile
import glob as gb
import librosa as lb
import os
import soundfile as sf

In [2]:
!unzip -uq "/content/drive/MyDrive/rawdess.zip" -d "/content/speech/" #unzipping the two files and placing them in a folder called speech
!unzip -uq "/content/drive/MyDrive/urbansound.zip" -d "/content/speech/"

audio_dir = '/content/speech/urbansound_8k'
noise_dir = '/content/speech/ravdess_rewritten_8k'

audio_files = gb.glob(os.path.join(audio_dir, '*.wav'))
noise_files = gb.glob(os.path.join(noise_dir, '*.wav'))

In [9]:
def newsound(speech_file, noise_file,output_path, sr=22050):
             speech, _ = lb.load(speech_file)
             noise, _ = lb.load(noise_file)


             if len(noise) > len(speech):
                noise = noise[:len(speech)]
                mixed = speech + noise
             else:
                  while len(noise) <= len(speech):
                        noise = np.concatenate((noise, noise))  # Double the length of noise until it exceeds target_length

                  noise=noise[:len(speech)]
                  mixed = speech + noise

             mixed = mixed / np.max(np.abs(mixed)) #prevents clipping(audio signal exceeds maximum amplitude)

             sf.write(output_path, mixed, 22050)





In [10]:
def main(audio_dir, noise_dir):
    output_dir = 'mixed_sounds'
    os.makedirs(output_dir, exist_ok=True)

    for audio_file, noise_file in zip(audio_files, noise_files): #iterates the audio files in pairs
        output_path = os.path.join(output_dir, os.path.basename(audio_file))
        newsound(audio_file, noise_file, output_path)

    with zipfile.ZipFile("/content/drive/MyDrive/mixed.zip", 'w') as zipf:  #zip all the files in the folder
        for root, _, files in os.walk(output_dir):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, output_dir))

In [11]:
main(audio_files,noise_files)